In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import h5py
from paltas.Utils.distribution_utils import geometric_average
import sys
sys.path.insert(0, '/Users/smericks/Desktop/StrongLensing/lens-npe/')
from Inference.network_hierarchical_inference import NetworkHierarchicalInference

This notebook demonstrates how to infer a population model from network predictions. 
We use the shifted test set for this example

## Load in Network Predictions ##

In [ ]:
# NPE model predictions 
npe_preds_path = 'from_zenodo/model_predictions/npe/diag/'

# SHIFTED SET
file_path = npe_preds_path+'narrow_predictions.h5'
h5f = h5py.File(file_path, 'r')
y_test_shifted = h5f.get('y_test').value
y_pred_shifted = h5f.get('y_pred').value
std_pred_shifted = h5f.get('std_pred').value
prec_pred_shifted = h5f.get('prec_pred').value
h5f.close()

# SHIFTED SET
y_pred_shifted_seq_list = np.load('from_zenodo/model_predictions/snpe/shifted/y_pred_list_epoch10.npy')
std_pred_shifted_seq_list = np.load('from_zenodo/model_predictions/snpe/shifted/std_pred_list_epoch10.npy')
prec_pred_shifted_seq_list = np.load('from_zenodo/model_predictions/snpe/shifted/prec_pred_list_epoch10.npy')


## Run HBI ##

In [ ]:
write_prefix = 'MYTEST/'

# note: center coordinates not included in HBI
train_mean_no_coords = [0.8,0.,0.,2.,0.,0]
train_scatter_no_coords = [0.15,0.12,0.12,0.2,0.2,0.2]

n_lenses = 20

# construct a list of y_pred, prec_pred to feed to HI object
y_pred_list_shifted = []
prec_pred_list_shifted = []
y_pred_list_shifted.append(y_pred_shifted[:n_lenses,:6])
# NOTE if diagonal, precision matrix can simply be truncated when marginalizing
prec_pred_list_shifted.append(prec_pred_shifted[:n_lenses,:6,:6])

# only m=2
y_pred_list_shifted.append(y_pred_shifted_seq_list[2,:n_lenses,:6])
prec_pred_list_shifted.append(prec_pred_shifted_seq_list[2,:n_lenses,:6,:6])

hi_quads = NetworkHierarchicalInference(train_mean_no_coords,train_scatter_no_coords,
        y_pred_list_shifted,prec_pred_list_shifted,
        hypermodel_type='fixed_param_no_coords',sigmas_log_uniform=False,
        n_emcee_samps=int(6e3))
_ = hi_quads.run_HI(write_prefix+'HI_fixed_param_no_coords.h5')